In [1]:
import nltk
import numpy as np
import random
import string

In [18]:
f=open('/Users/amipatel/Desktop/kb.txt','r',errors = 'ignore')
raw=f.read()
raw=raw.lower()
raw=' '.join(raw.split())

sent_tokens = nltk.sent_tokenize(raw)
word_tokens = nltk.word_tokenize(raw)

In [9]:
sent_tokens[:2]

['people tend to lose weight quickly on low carb diets because they restrict their calories to about 1,200 a day.',
 "this is about 500 calories per day for women, and 600 for men.you can choose whichever two days of the week you prefer, as long as there is at least one non fasting day in between them.one common way of planning the week is to fast on mondays and thursdays, with two or three small meals, then eat normally for the rest of the week.it's important to emphasize that eating normally does not mean you can eat anything."]

In [10]:
inputs = ("hello", "hi", "greetings", "what up", "what's up","hey",)
response1 = ["hi", "hey", "hi there", "hello", "hi, I'm good"]

def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in inputs:
            return random.choice(response1)

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [32]:
from textblob import TextBlob
import re

In [70]:
def find_prp(s):
    response = None
    
    for w,POS in s.pos_tags:
        if w == 'you':
            response = 'I'
        if w == 'i':
            response = 'You'
            
    return response

def find_np(s):
#     s = TextBlob(s)
    if not s.noun_phrases:
        return None
    else:
        return s.noun_phrases

def find_parts_of_speech(parsed):
    for s in parsed.sentences:
        prp_response = find_prp(s)
        np_response = find_np(s)
        
    return prp_response, np_response
    
def response2(user):
    user = user.lower()
    punctuations = '''!()-[]{};:"\,<>/?@#$%^&*_~'''
    for x in user: 
        if x in punctuations: 
            user = user.replace(x, " ") 
    content = re.sub(r'[^a-z0-9 .]+', ' ',user)
    parsed = TextBlob(content)
    prp, np = find_parts_of_speech(parsed)
    rest = 'Sure, '
    for w,POS in parsed.pos_tags:
        if w == 'you'or w == 'i':
            rest += prp
        else:
            rest += ' ' + w
            
    return rest
    

In [12]:
def response(user_response):
    bot_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer()
    tfidf = TfidfVec.fit_transform(sent_tokens)
    similar_vector_vals = cosine_similarity(tfidf[-1], tfidf)
    
    # the second last sentence will have the highest cosine similarity
    sentence_number=similar_vector_vals.argsort()[0][-2]
    
    # Flatten to check if the cosine similarity is 0 or not
    flat = similar_vector_vals.flatten()
    flat.sort()
    actual_tfidf = flat[-2]
    if(actual_tfidf==0):
        bot_response=bot_response+"I am sorry! I don't understand you"
        return bot_response
    else:
        bot_response = bot_response+sent_tokens[sentence_number]
        return bot_response

In [71]:
user_response = input()
print(response2(user_response))

I have a happy face
Sure, You have a happy face


In [14]:
Bye = False
print("DIETO: My name is DIETO. I will answer your queries about diets. If you want to exit, type Bye!")
while(Bye is not True):
    print("YOU: ", end = '')
    user_response = input()
    user_response=user_response.lower()
    if(user_response!='bye'):
        if(user_response=='thanks' or user_response=='thank you' ):
            print("DIETO: You are welcome.")
        else:
            if(greeting(user_response)!=None):
                print("DIETO: "+greeting(user_response))
            else:
                print("DIETO: ",end="")
                print(response(user_response))
                sent_tokens.remove(user_response)
    else:
        Bye = True
        print("DIETO: Bye! take care..")

DIETO: My name is DIETO. I will answer your queries about diets. If you want to exit, type Bye!
YOU: bye
DIETO: Bye! take care..
